Created by DARIO MAMELI

In [1]:
!pip install -r requirements.txt
!pip install ultralytics==8.0.20

In [2]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2
import os
import shutil

## Segmentation

Perform the segmentation and save the binary segments

In [3]:
# Choose model
model = YOLO("yolov8s-seg.pt")

In [4]:
folder = r"../Sport_scene_dataset/Images" # Dataset location
output_folder = r"../runs/segment/predict/masks"  # Where to save the masks
output_folder_images = r"../runs/segment/predict/images" # Where to save the segmented images

# Recreate output directories
shutil.rmtree(output_folder)
os.makedirs(output_folder)
shutil.rmtree(output_folder_images)
os.makedirs(output_folder_images)

# Iterate through each file in the folder
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    
    # Check if the item is a file (not a subfolder)
    if os.path.isfile(file_path):
        print(f'Found file: {filename}')
    
        # Read the image for masks creation
        img = cv2.imread(file_path)
    
        # Run inference
        results = model.predict(source=file_path, retina_masks=True, classes=0)
    
        imageName = filename[:-4]

        # Get the masks for people (class 0)
        if results[0].masks is None:
            continue
        masks = results[0].masks.data
        people_masks = masks if len(masks) > 0 else None
        
        if people_masks is not None:
            # Convert the tensor to a NumPy array
            people_masks_np = people_masks.cpu().numpy()
                        
            i = 0
            for instance_mask in people_masks_np:
                # Get each instance and save it after resize
                instance_mask_resized = cv2.resize(instance_mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)
                instance_mask_toSave = (instance_mask_resized > 0).astype(np.uint8) * 255
                output_file_path = os.path.join(output_folder, f"{imageName}_mask_{i}.png")
                cv2.imwrite(output_file_path, instance_mask_toSave)
                i = i+1
                
                # Create a BGR image (3 channels)
                mask = np.zeros_like(img) # Initialized to 0
                mask[:,:,2] = instance_mask_toSave # Saved the values of the instance in the red channel
                
                # Overlay the red mask on the image
                img = cv2.addWeighted(img, 1, mask, 0.8, 0)
                
            # Save the overlaid image
            output_file_path = os.path.join(output_folder_images, f"{imageName}_segmented.jpg")
            cv2.imwrite(output_file_path, img)
            print("File saved at ", str(output_file_path))
                
        else:
            print(f'No mask found for {filename}')    

Found file: im09.jpg


Ultralytics YOLOv8.0.20  Python-3.9.13 torch-2.0.1+cpu CPU
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients, 42.6 GFLOPs


File saved at  ../runs/segment/predict/images\im09_segmented.jpg
Found file: im1.jpg
File saved at  ../runs/segment/predict/images\im1_segmented.jpg
Found file: im10.jpg
File saved at  ../runs/segment/predict/images\im10_segmented.jpg
Found file: im11.jpg
File saved at  ../runs/segment/predict/images\im11_segmented.jpg
Found file: im12.jpg
File saved at  ../runs/segment/predict/images\im12_segmented.jpg
Found file: im13.jpg
File saved at  ../runs/segment/predict/images\im13_segmented.jpg
Found file: im14.jpg
File saved at  ../runs/segment/predict/images\im14_segmented.jpg
Found file: im15.jpg
File saved at  ../runs/segment/predict/images\im15_segmented.jpg
Found file: im2.jpg
File saved at  ../runs/segment/predict/images\im2_segmented.jpg
Found file: im3.jpg
File saved at  ../runs/segment/predict/images\im3_segmented.jpg
Found file: im4.jpg
File saved at  ../runs/segment/predict/images\im4_segmented.jpg
Found file: im5.jpg
File saved at  ../runs/segment/predict/images\im5_segmented.jpg